In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
tf.keras.backend.clear_session()

In [5]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [6]:
subject = 'Ceará - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [7]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_CE.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Ano,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - IDH Longevidade,Ceará - IDH Educacao,Bahia - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Ceará - IDH,Ceará - Produção de Cimento (t),Ceará - Consumo de Cimento (t)
0,2008,7.430439e+07,4.781538e+06,7.686809,6.447530e+07,0.788078,0.686465,0.659869,5.483335e+08,2.546644e+09,...,19.530149,1008.923849,1487.946810,27.057592,21.627565,73.260756,2.648291,0.705344,1697.104012,1328.591000
1,2009,7.487461e+07,4.827502e+06,7.698375,6.456035e+07,0.790273,0.691475,0.660519,5.302764e+08,2.679945e+09,...,19.393252,1008.908015,1398.714107,27.170444,21.546547,72.588532,2.613315,0.708210,1747.798488,1437.485000
2,2010,7.682885e+07,4.851802e+06,7.744071,6.510367e+07,0.791872,0.696107,0.660850,5.046425e+08,2.891930e+09,...,19.495931,1008.929298,1352.609172,27.206178,21.476514,72.156276,2.630976,0.710635,1745.258962,1794.240000
3,2011,7.858864e+07,4.848876e+06,7.773847,6.550376e+07,0.792885,0.700356,0.660869,4.854622e+08,3.091988e+09,...,18.136683,1009.081267,1353.481052,27.247003,21.498658,72.034757,2.712631,0.712630,1751.304575,1717.740000
4,2012,8.015440e+07,4.829775e+06,7.791383,6.580570e+07,0.793321,0.704333,0.660403,4.867661e+08,3.189506e+09,...,17.755013,1009.208938,1324.118308,27.303325,21.513635,71.799537,2.739656,0.714175,1769.225958,1903.741000
5,2013,8.160149e+07,4.793349e+06,7.811016,6.610515e+07,0.793098,0.707444,0.659936,5.225848e+08,3.093872e+09,...,18.955957,1009.427497,1294.208198,27.279320,21.416356,71.445993,2.723080,0.715273,1802.564432,2065.389000
6,2014,8.275393e+07,4.756888e+06,7.821411,6.627704e+07,0.792439,0.710814,0.659347,5.676413e+08,2.779340e+09,...,18.599712,1009.682618,1313.782408,27.429527,21.316386,70.443006,2.652161,0.716129,1936.472431,1964.413143
7,2015,8.340186e+07,4.705636e+06,7.807860,6.621518e+07,0.791476,0.713466,0.657960,5.179392e+08,2.561731e+09,...,18.179780,1009.682825,1316.471785,27.559921,21.342508,70.089489,2.538890,0.716639,2117.547255,2037.703714
8,2016,8.386853e+07,4.627836e+06,7.802392,6.619932e+07,0.802758,0.730642,0.662422,4.524144e+08,2.516859e+09,...,19.751583,1009.575611,1278.441481,27.460532,21.334656,70.525466,2.458740,0.729111,2241.119301,2146.547143
9,2017,8.401644e+07,4.573568e+06,7.799909,6.618240e+07,0.801917,0.732700,0.662461,4.213689e+08,2.497710e+09,...,21.401573,1009.628585,1268.231186,27.368964,21.410344,71.183860,2.400073,0.729757,2243.972991,2035.158000


In [8]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - IDH Longevidade,Ceará - IDH Educacao,Bahia - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Ceará - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Ceará - IDH,Ceará - Produção de Cimento (t)
0,7.430439e+07,4.781538e+06,7.686809,6.447530e+07,0.788078,0.686465,0.659869,5.483335e+08,2.546644e+09,1.874883e+10,...,8.024047,19.530149,1008.923849,1487.946810,27.057592,21.627565,73.260756,2.648291,0.705344,1697.104012
1,7.487461e+07,4.827502e+06,7.698375,6.456035e+07,0.790273,0.691475,0.660519,5.302764e+08,2.679945e+09,2.055015e+10,...,8.052145,19.393252,1008.908015,1398.714107,27.170444,21.546547,72.588532,2.613315,0.708210,1747.798488
2,7.682885e+07,4.851802e+06,7.744071,6.510367e+07,0.791872,0.696107,0.660850,5.046425e+08,2.891930e+09,2.342247e+10,...,8.017491,19.495931,1008.929298,1352.609172,27.206178,21.476514,72.156276,2.630976,0.710635,1745.258962
3,7.858864e+07,4.848876e+06,7.773847,6.550376e+07,0.792885,0.700356,0.660869,4.854622e+08,3.091988e+09,2.604265e+10,...,8.006912,18.136683,1009.081267,1353.481052,27.247003,21.498658,72.034757,2.712631,0.712630,1751.304575
4,8.015440e+07,4.829775e+06,7.791383,6.580570e+07,0.793321,0.704333,0.660403,4.867661e+08,3.189506e+09,2.708753e+10,...,9.818854,17.755013,1009.208938,1324.118308,27.303325,21.513635,71.799537,2.739656,0.714175,1769.225958
5,8.160149e+07,4.793349e+06,7.811016,6.610515e+07,0.793098,0.707444,0.659936,5.225848e+08,3.093872e+09,2.523394e+10,...,10.221864,18.955957,1009.427497,1294.208198,27.279320,21.416356,71.445993,2.723080,0.715273,1802.564432
6,8.275393e+07,4.756888e+06,7.821411,6.627704e+07,0.792439,0.710814,0.659347,5.676413e+08,2.779340e+09,1.954361e+10,...,10.652422,18.599712,1009.682618,1313.782408,27.429527,21.316386,70.443006,2.652161,0.716129,1936.472431
7,8.340186e+07,4.705636e+06,7.807860,6.621518e+07,0.791476,0.713466,0.657960,5.179392e+08,2.561731e+09,1.327596e+10,...,11.020421,18.179780,1009.682825,1316.471785,27.559921,21.342508,70.089489,2.538890,0.716639,2117.547255
8,8.386853e+07,4.627836e+06,7.802392,6.619932e+07,0.802758,0.730642,0.662422,4.524144e+08,2.516859e+09,1.011370e+10,...,11.073199,19.751583,1009.575611,1278.441481,27.460532,21.334656,70.525466,2.458740,0.729111,2241.119301
9,8.401644e+07,4.573568e+06,7.799909,6.618240e+07,0.801917,0.732700,0.662461,4.213689e+08,2.497710e+09,8.456309e+09,...,10.712887,21.401573,1009.628585,1268.231186,27.368964,21.410344,71.183860,2.400073,0.729757,2243.972991


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     1437.485000
1     1794.240000
2     1717.740000
3     1903.741000
4     2065.389000
5     1964.413143
6     2037.703714
7     2146.547143
8     2035.158000
9     1758.900000
10    1701.230000
11    1864.364000
12    2127.104000
13    1966.646000
14            NaN
Name: Ceará - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - IDH Longevidade,Ceará - IDH Educacao,Bahia - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Ceará - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Ceará - IDH,Ceará - Produção de Cimento (t)
0,-1.825466,0.487152,-2.082162,-1.966996,-1.432506,-1.530955,-0.598881,1.204752,-0.990234,0.166091,...,-1.370027,-0.037551,-1.397395,2.361873,-1.968241,1.645394,1.854362,0.575704,-1.479238,-1.125640
1,-1.660185,0.916292,-1.805814,-1.829234,-0.965438,-1.228499,-0.056326,0.820657,-0.546688,0.428911,...,-1.347480,-0.135767,-1.449574,1.110791,-1.102522,0.815692,1.097108,0.316306,-1.145473,-0.921252
2,-1.093737,1.143174,-0.713987,-0.949131,-0.625247,-0.948873,0.220138,0.275392,0.158672,0.847995,...,-1.375288,-0.062101,-1.379437,0.464379,-0.828394,0.098490,0.610176,0.447285,-0.863001,-0.931491
3,-0.583651,1.115853,-0.002554,-0.301050,-0.409551,-0.692386,0.236503,-0.132596,0.824346,1.230291,...,-1.383777,-1.037284,-0.878631,0.476603,-0.515214,0.325264,0.473287,1.052873,-0.630588,-0.907116
4,-0.129809,0.937515,0.416452,0.188047,-0.316832,-0.452299,-0.153228,-0.104860,1.148829,1.382743,...,0.070207,-1.311110,-0.457901,0.064924,-0.083159,0.478640,0.208314,1.253302,-0.450685,-0.834861
5,0.289639,0.597428,0.885541,0.673111,-0.364332,-0.264486,-0.542565,0.657046,0.830615,1.112297,...,0.393601,-0.449501,0.262347,-0.354429,-0.267309,-0.517588,-0.189950,1.130368,-0.322806,-0.700448
6,0.623680,0.257006,1.133921,0.951549,-0.504446,-0.061023,-1.033902,1.615453,-0.215960,0.282052,...,0.739100,-0.705087,1.103081,-0.079990,0.884973,-1.541380,-1.319806,0.604407,-0.223096,-0.160562
7,0.811486,-0.221506,0.810123,0.851339,-0.709430,0.099028,-2.192189,0.558230,-0.940033,-0.632425,...,1.034399,-1.006364,1.103764,-0.042283,1.885254,-1.273864,-1.718039,-0.235662,-0.163608,0.569490
8,0.946754,-0.947874,0.679480,0.825655,1.691026,1.135918,1.532636,-0.835563,-1.089341,-1.093812,...,1.076751,0.121315,0.750445,-0.575485,1.122822,-1.354272,-1.226915,-0.830082,1.288969,1.067704
9,0.989625,-1.454550,0.620159,0.798254,1.512250,1.260177,1.564684,-1.495939,-1.153056,-1.335633,...,0.787620,1.305088,0.925018,-0.718638,0.420374,-0.579156,-0.485241,-1.265186,1.364260,1.079209


In [11]:
reshaped_train_input = dfToInputRNN(train_input)

In [12]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 0.71095889, -1.3078241 , -0.17545801,  0.22487172,
          0.88121488,  1.20194732,  0.46194009, -1.0453709 ,
          1.21859064, -0.99976018,  0.79428371,  1.15575749,
         -0.52559858,  1.30260559, -0.55253046, -0.25130359,
          1.38787176,  1.47154451, -1.80649616,  1.09296291,
          0.94281138,  0.52836084,  0.61258103,  0.87656257,
          1.47801536,  1.71982086,  1.28221181,  1.39086472,
          1.58209844,  1.08872091,  0.55212962, -1.81398118,
          1.44113674,  1.11191094,  1.30470835,  0.86390516,
         -1.67365968, -1.70738939, -1.55813196, -0.48498301,
         -0.75714639,  0.40770893, -0.27394734,  2.02323219,
          1.2908951 ,  1.70465513,  1.81148084,  1.39968653,
         -1.36191442,  1.28024593,  1.11054795,  0.12662799,
          1.40414112,  1.40468999,  1.45643959,  1.58509336,
          1.32283157,  1.30222047,  1.55138602,  1.91801602,
         -0.77489784,  2.32098884, -1.1942541 , -1.17081042,
          0.65873367,  1

In [13]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     1437.485000
1     1794.240000
2     1717.740000
3     1903.741000
4     2065.389000
5     1964.413143
6     2037.703714
7     2146.547143
8     2035.158000
9     1758.900000
10    1701.230000
11    1864.364000
Name: Ceará - Consumo de Cimento (t), dtype: float64

In [14]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    1864.364
Name: Ceará - Consumo de Cimento (t), dtype: float64

In [15]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - IDH Longevidade,Ceará - IDH Educacao,Bahia - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Ceará - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Ceará - IDH,Ceará - Produção de Cimento (t)
12,0.373891,-1.122598,-0.818152,-0.27511,0.400623,1.168563,-0.47095,1.108405,2.635823,-0.189959,...,0.613586,1.025085,-0.211439,-1.463016,0.503155,1.032998,0.281873,-1.020017,1.013884,2.067146


In [16]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.37389055, -1.12259774, -0.81815199, -0.27511001,
          0.40062263,  1.16856259, -0.47095012,  1.10840491,
          2.63582309, -0.18995911,  0.72640489,  2.66833308,
         -0.032632  ,  0.98802435,  0.24980494, -0.08955808,
          1.85434394,  2.02706568, -2.21150703,  1.42938591,
          0.85132668,  0.14092386,  0.30902324,  1.00267904,
          2.27970137,  1.11272593,  0.61007902,  0.89064538,
          1.42231438,  1.02650562,  1.47127593, -1.53201473,
          1.73503858,  1.66610613,  1.0456894 ,  0.55289259,
         -1.81216593, -2.10342296, -2.55377478,  0.41940628,
         -1.75756445,  0.241023  , -0.98659345,  1.52184916,
          1.01398607,  1.49290967,  1.40764092,  1.87930411,
         -2.51892072,  2.83356214,  1.58620924, -0.02524128,
          1.54231542,  1.5176119 ,  1.7355269 ,  1.87941982,
          1.54198129,  1.51276712,  1.83795594,  2.62210693,
          1.04993174,  0.96476493, -1.48277861, -2.26044738,
          0.61358587,  1

In [17]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    2127.104
Name: Ceará - Consumo de Cimento (t), dtype: float64

In [18]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh')),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        batch_size=train_input.shape[0],
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [19]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [29]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[1976215282, 3981205573, 1792484424, 1999006000, 885968896, 919977224, 3877108982, 1606452188, 1726789711, 4262801448, 3511153179, 3009154811, 1100657578, 1377260543, 810896875, 1316406898, 3607542125, 1532112756, 91341726, 321404160, 4080566076, 4038974527, 502311544, 3588647321, 3062422894, 2864548474, 1305492252, 1770749870, 4032400976, 3650908919, 504600262, 841679953, 3303169292, 4040379819, 3394157445, 4067921542, 1226915339, 2618453650, 2606292394, 2314752643, 2355776176, 1672299454, 3406871489, 806793367, 207886676, 2177434586, 402768002, 2934967872, 3040374963, 1601292354, 3025286860, 3476943285, 2357315899, 3936019327, 2756192705, 3374027666, 3351189240, 3077839674, 2556100124, 4258769714, 1191774235, 317256709, 2810949128, 1540530275, 1201412723, 2106400566, 4256649538, 252758768, 3160010292, 2136573048, 1643480115, 3334811598, 2744252435, 2684058023, 2108832491, 833615268, 3882047934, 2145055639, 2037900775, 3414395070, 3490538487, 856532854, 1959019974, 656043925, 37407501

loss: 7395.51318359375


Step: 91 ___________________________________________
loss: 5207.16259765625


Step: 92 ___________________________________________
loss: 5593.826171875


Step: 93 ___________________________________________
loss: 7875.72802734375


Step: 94 ___________________________________________
loss: 5216.31787109375


Step: 95 ___________________________________________
loss: 4459.388671875


Step: 96 ___________________________________________
loss: 8695.9677734375


Step: 97 ___________________________________________
loss: 4433.1328125


Step: 98 ___________________________________________
loss: 6592.02880859375


Step: 99 ___________________________________________
loss: 7299.81201171875


final_seed: 4262801448


In [30]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 2s 2s/step - loss: 3529199.7500
Epoch 2/10000
1/1 [==============================] - 0s 5ms/step - loss: 3371487.2500
Epoch 3/10000
1/1 [==============================] - 0s 4ms/step - loss: 1242347.5000
Epoch 4/10000
1/1 [==============================] - 0s 4ms/step - loss: 714174.6875
Epoch 5/10000
1/1 [==============================] - 0s 4ms/step - loss: 1618913.8750
Epoch 6/10000
1/1 [==============================] - 0s 4ms/step - loss: 165923.4375
Epoch 7/10000
1/1 [==============================] - 0s 4ms/step - loss: 291238.4062
Epoch 8/10000
1/1 [==============================] - 0s 4ms/step - loss: 97579.4688
Epoch 9/10000
1/1 [==============================] - 0s 4ms/step - loss: 94880.0859
Epoch 10/10000
1/1 [==============================] - 0s 4ms/step - loss: 48233.8789
Epoch 11/10000
1/1 [==============================] - 0s 4ms/step - loss: 51148.0469
Epoch 12/10000
1/1 [==============================] - 0s 4ms/ste

1/1 [==============================] - 0s 4ms/step - loss: 22049.5332
Epoch 98/10000
1/1 [==============================] - 0s 4ms/step - loss: 19680.5605
Epoch 99/10000
1/1 [==============================] - 0s 4ms/step - loss: 50483.3242
Epoch 100/10000
1/1 [==============================] - 0s 4ms/step - loss: 27691.4062
Epoch 101/10000
1/1 [==============================] - 0s 4ms/step - loss: 44446.1875
Epoch 102/10000
1/1 [==============================] - 0s 4ms/step - loss: 25260.7031
Epoch 103/10000
1/1 [==============================] - 0s 4ms/step - loss: 21820.3535
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 48461.8438
Epoch 105/10000
1/1 [==============================] - 0s 4ms/step - loss: 22617.4512
Epoch 106/10000
1/1 [==============================] - 0s 4ms/step - loss: 24025.1465
Epoch 107/10000
1/1 [==============================] - 0s 3ms/step - loss: 35692.8789
Epoch 108/10000
1/1 [==============================] - 0s 4ms/step - los

1/1 [==============================] - 0s 4ms/step - loss: 49876.3555
Epoch 193/10000
1/1 [==============================] - 0s 4ms/step - loss: 33062.9883
Epoch 194/10000
1/1 [==============================] - 0s 4ms/step - loss: 35112.4570
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 48523.5742
Epoch 196/10000
1/1 [==============================] - 0s 4ms/step - loss: 19700.2773
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 31366.6504
Epoch 198/10000
1/1 [==============================] - 0s 4ms/step - loss: 22976.0078
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 29082.2012
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 39105.5977
Epoch 201/10000
1/1 [==============================] - 0s 4ms/step - loss: 38450.6680
Epoch 202/10000
1/1 [==============================] - 0s 4ms/step - loss: 26451.4844
Epoch 203/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 53607.1133
Epoch 288/10000
1/1 [==============================] - 0s 4ms/step - loss: 39319.6172
Epoch 289/10000
1/1 [==============================] - 0s 4ms/step - loss: 26143.9746
Epoch 290/10000
1/1 [==============================] - 0s 4ms/step - loss: 35300.4688
Epoch 291/10000
1/1 [==============================] - 0s 4ms/step - loss: 25030.2051
Epoch 292/10000
1/1 [==============================] - 0s 4ms/step - loss: 43452.8789
Epoch 293/10000
1/1 [==============================] - 0s 4ms/step - loss: 24553.0781
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 19852.4180
Epoch 295/10000
1/1 [==============================] - 0s 4ms/step - loss: 40983.6289
Epoch 296/10000
1/1 [==============================] - 0s 4ms/step - loss: 24031.7988
Epoch 297/10000
1/1 [==============================] - 0s 4ms/step - loss: 20380.7129
Epoch 298/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 44451.7461
Epoch 383/10000
1/1 [==============================] - 0s 4ms/step - loss: 26195.4668
Epoch 384/10000
1/1 [==============================] - 0s 4ms/step - loss: 37432.9336
Epoch 385/10000
1/1 [==============================] - 0s 4ms/step - loss: 37758.5508
Epoch 386/10000
1/1 [==============================] - 0s 4ms/step - loss: 34583.5977
Epoch 387/10000
1/1 [==============================] - 0s 4ms/step - loss: 41573.2070
Epoch 388/10000
1/1 [==============================] - 0s 4ms/step - loss: 29591.7559
Epoch 389/10000
1/1 [==============================] - 0s 4ms/step - loss: 49275.6445
Epoch 390/10000
1/1 [==============================] - 0s 4ms/step - loss: 22037.1660
Epoch 391/10000
1/1 [==============================] - 0s 4ms/step - loss: 39439.2969
Epoch 392/10000
1/1 [==============================] - 0s 4ms/step - loss: 23679.5801
Epoch 393/10000
1/1 [==============================] - 0s 3ms/step - l

...

1/1 [==============================] - 0s 4ms/step - loss: 37919.2070
Epoch 668/10000
1/1 [==============================] - 0s 3ms/step - loss: 21011.9316
Epoch 669/10000
1/1 [==============================] - 0s 4ms/step - loss: 27153.5020
Epoch 670/10000
1/1 [==============================] - 0s 4ms/step - loss: 19015.2051
Epoch 671/10000
1/1 [==============================] - 0s 4ms/step - loss: 33229.6719
Epoch 672/10000
1/1 [==============================] - 0s 4ms/step - loss: 17567.0156
Epoch 673/10000
1/1 [==============================] - 0s 4ms/step - loss: 25175.5938
Epoch 674/10000
1/1 [==============================] - 0s 4ms/step - loss: 29713.7031
Epoch 675/10000
1/1 [==============================] - 0s 4ms/step - loss: 23111.0254
Epoch 676/10000
1/1 [==============================] - 0s 4ms/step - loss: 19516.8516
Epoch 677/10000
1/1 [==============================] - 0s 4ms/step - loss: 31306.7969
Epoch 678/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 14611.3662
Epoch 763/10000
1/1 [==============================] - 0s 4ms/step - loss: 29160.9219
Epoch 764/10000
1/1 [==============================] - 0s 4ms/step - loss: 12643.4189
Epoch 765/10000
1/1 [==============================] - 0s 4ms/step - loss: 28597.7441
Epoch 766/10000
1/1 [==============================] - 0s 4ms/step - loss: 32244.1426
Epoch 767/10000
1/1 [==============================] - 0s 4ms/step - loss: 20512.9141
Epoch 768/10000
1/1 [==============================] - 0s 4ms/step - loss: 15137.0938
Epoch 769/10000
1/1 [==============================] - 0s 4ms/step - loss: 31220.8379
Epoch 770/10000
1/1 [==============================] - 0s 4ms/step - loss: 21066.3652
Epoch 771/10000
1/1 [==============================] - 0s 4ms/step - loss: 21064.2598
Epoch 772/10000
1/1 [==============================] - 0s 4ms/step - loss: 13008.8945
Epoch 773/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 20641.4121
Epoch 858/10000
1/1 [==============================] - 0s 4ms/step - loss: 18365.2637
Epoch 859/10000
1/1 [==============================] - 0s 4ms/step - loss: 20324.8398
Epoch 860/10000
1/1 [==============================] - 0s 3ms/step - loss: 10717.3867
Epoch 861/10000
1/1 [==============================] - 0s 4ms/step - loss: 9974.9346
Epoch 862/10000
1/1 [==============================] - 0s 4ms/step - loss: 31535.8770
Epoch 863/10000
1/1 [==============================] - 0s 3ms/step - loss: 14711.6172
Epoch 864/10000
1/1 [==============================] - 0s 3ms/step - loss: 27606.2441
Epoch 865/10000
1/1 [==============================] - 0s 4ms/step - loss: 32207.8535
Epoch 866/10000
1/1 [==============================] - 0s 4ms/step - loss: 31904.6465
Epoch 867/10000
1/1 [==============================] - 0s 4ms/step - loss: 32463.1230
Epoch 868/10000
1/1 [==============================] - 0s 4ms/step - lo

1/1 [==============================] - 0s 4ms/step - loss: 19916.3027
Epoch 953/10000
1/1 [==============================] - 0s 4ms/step - loss: 15392.0049
Epoch 954/10000
1/1 [==============================] - 0s 3ms/step - loss: 21880.2656
Epoch 955/10000
1/1 [==============================] - 0s 4ms/step - loss: 23591.5215
Epoch 956/10000
1/1 [==============================] - 0s 4ms/step - loss: 25826.2422
Epoch 957/10000
1/1 [==============================] - 0s 4ms/step - loss: 34187.1680
Epoch 958/10000
1/1 [==============================] - 0s 4ms/step - loss: 28594.1562
Epoch 959/10000
1/1 [==============================] - 0s 3ms/step - loss: 15639.9170
Epoch 960/10000
1/1 [==============================] - 0s 4ms/step - loss: 16847.1523
Epoch 961/10000
1/1 [==============================] - 0s 4ms/step - loss: 10336.6377
Epoch 962/10000
1/1 [==============================] - 0s 3ms/step - loss: 36095.4727
Epoch 963/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 27025.4199
Epoch 1047/10000
1/1 [==============================] - 0s 4ms/step - loss: 21850.5469
Epoch 1048/10000
1/1 [==============================] - 0s 4ms/step - loss: 26677.4785
Epoch 1049/10000
1/1 [==============================] - 0s 4ms/step - loss: 36861.2109
Epoch 1050/10000
1/1 [==============================] - 0s 4ms/step - loss: 37381.1758
Epoch 1051/10000
1/1 [==============================] - 0s 4ms/step - loss: 21497.1738
Epoch 1052/10000
1/1 [==============================] - 0s 4ms/step - loss: 16749.1992
Epoch 1053/10000
1/1 [==============================] - 0s 4ms/step - loss: 32144.8223
Epoch 1054/10000
1/1 [==============================] - 0s 3ms/step - loss: 34232.9570
Epoch 1055/10000
1/1 [==============================] - 0s 4ms/step - loss: 19913.7559
Epoch 1056/10000
1/1 [==============================] - 0s 4ms/step - loss: 32610.1484
Epoch 1057/10000
1/1 [==============================] - 0s 4

In [31]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 1s/step
[2127.104](test_target) - [1858.9738](prediction) = 268.1302451171873


In [32]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.12605413045962366

In [33]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [34]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - IDH Longevidade,Ceará - IDH Educacao,Bahia - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Ceará - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Ceará - IDH,Ceará - Produção de Cimento (t)
0,-0.954262,-1.329710,-0.928006,-0.854355,-1.283635,-1.240439,-1.334114,1.148205,-1.122043,-1.121418,...,-0.477724,0.980535,0.383452,1.332476,-1.376174,1.253331,1.303231,1.220695,-1.257275,-1.412816
1,-0.426771,0.247838,-0.460175,-0.548813,0.127803,0.032016,0.260725,0.140893,-0.184474,-0.185492,...,1.391613,-1.372828,-1.370591,-0.255896,0.405937,-0.059328,-0.176019,-1.228755,0.067883,0.760852
2,1.381033,1.081872,1.388180,1.403168,1.155832,1.208423,1.073389,-1.289098,1.306518,1.306910,...,-0.913889,0.392294,0.987139,-1.076580,0.970237,-1.194003,-1.127212,0.008060,1.189392,0.651963


train_target:


,Ceará - Consumo de Cimento (t)
0,1437.485
1,1794.240
2,1717.740


test_input:


...

,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - IDH Longevidade,Ceará - IDH Educacao,Bahia - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Ceará - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Ceará - IDH,Ceará - Produção de Cimento (t)
12,0.373891,-1.122598,-0.818152,-0.27511,0.400623,1.168563,-0.47095,1.108405,2.635823,-0.189959,...,0.613586,1.025085,-0.211439,-1.463016,0.503155,1.032998,0.281873,-1.020017,1.013884,2.067146


test_target:


,Ceará - Consumo de Cimento (t)
12,2127.104


1/1 [==============================] - 1s 592ms/step
Error: 268.1302451171873




[1661.0357666015625,
 1755.7098388671875,
 1974.9970703125,
 1901.515869140625,
 1992.58544921875,
 2035.0865478515625,
 2111.14306640625,
 1904.3853759765625,
 1699.8721923828125,
 1858.9737548828125]

In [35]:
display(targets)
display(predictions)

[1903.741,
 2065.389,
 1964.4131428571427,
 2037.703714285714,
 2146.5471428571427,
 2035.158,
 1758.9,
 1701.23,
 1864.364,
 2127.104]

[1661.0357666015625,
 1755.7098388671875,
 1974.9970703125,
 1901.515869140625,
 1992.58544921875,
 2035.0865478515625,
 2111.14306640625,
 1904.3853759765625,
 1699.8721923828125,
 1858.9737548828125]

In [36]:
mae = mean_absolute_error(predictions, targets)
mae

184.1209808035714

In [37]:
porcentage = mae/np.mean(targets)
porcentage

0.09391747364952086